# Tutorial

This is a brief tutorial of basic Metagraph usage.

First, we import Metagraph:

In [ ]:
import metagraph as mg

## Inspecting Types and Available Algorithms

The default resolver automatically pulls in all registered Metagraph plugins.

In [ ]:
res = mg.resolver

A hierarchy of available types is automatically added as properties on `res`.

In [ ]:
dir(res.types)

Two important concepts in Metagraph are abstract types and concrete types. 

Abstract types describe a generic kind of data container with potentially many equivalent representations.

Concrete types describe a specific data object which fits under the abstract type category.

One can think of abstract types as data container specifications and concrete types as implementations of those specifications.

For each abstract type, there are several concrete types.

Within a single abstract type, all concrete types are able to represent equivalent data, but in a different format or data structure.

Here we show the concrete types which represent `Graphs`:

In [ ]:
dir(res.types.Graph)

Algorithms are also listed under `res.algos` and grouped by categories.

In [ ]:
dir(res.algos)

In [ ]:
dir(res.algos.traversal)

## Example Usage

Let's see how to use Metagraph by first constructing a graph from an edge list.

Begin with an input csv file representing an edge list and weights.

In [ ]:
data = """
Source,Destination,Weight
0,1,4
0,3,2
0,4,7
1,3,3
1,4,5
2,4,5
2,5,2
2,6,8
3,4,1
4,7,4
5,6,4
5,7,6
"""

Read in the csv file and convert to a Pandas `DataFrame`.

In [ ]:
import pandas as pd
import io
csv_file = io.StringIO(data)
df = pd.read_csv(csv_file)

This `DataFrame` represents a graph’s edges, but Metagraph doesn’t know that yet. To use the `DataFrame` within Metagraph, we first need to convert it into an `EdgeMap`.

A `PandasEdgeMap` takes a `DataFrame` plus the labels of the columns representing source, destination, and weight. With these, Metagraph will know how to interpret the `DataFrame` as an `EdgeMap`.

In [ ]:
em = res.wrappers.EdgeMap.PandasEdgeMap(df, 'Source', 'Destination', 'Weight', is_directed=False)
em.value

## Convert EdgeMap to a Graph

`Graphs` and `EdgeMaps` have many similarities, but `Graphs` are more powerful. `Graphs` can have weights on the nodes, not just on the edges. `Graphs` can also have isolate nodes (nodes with no edges), which `EdgeMaps` cannot have.

Most Metagraph algorithms take a `Graph` as input, so we will convert our `PandasEdgeMap` into a `Graph`. In this case, it will become a `NetworkXGraph`.

In [ ]:
g = res.algos.util.graph.build(em)
g

In [ ]:
g.value.edges(data=True)

## Translate to other Graph formats

Because Metagraph knows how to interpret `g` as a `Graph`, we can easily convert it other `Graph` formats.

Let's convert it to a `ScipyGraph`. This format stores the edges and weights in a `ScipyEdgeMap` and any node weights in a `NumpyNodeMap`.

In [ ]:
g2 = res.translate(g, res.wrappers.Graph.ScipyGraph)
g2

The `ScipyEdgeMap` is accessed using `g2.edges`. Within the `EdgeMap`, the underlying scipy.sparse matrix is accessed using `.value`.

We can verify the weighs and edges by inspecting the sparse adjacency matrix directly.

In [ ]:
g2.edges.value.toarray()

We can also convert `g` into an adjacency matrix representation using a `GrblasGraph`. This also stores the edges and nodes separately.

In [ ]:
g3 = res.translate(g, res.types.Graph.GrblasGraphType)
g3

In [ ]:
g3.edges.value

We can also visualize the graph.

In [ ]:
import grblas
grblas.io.draw(g3.edges.value)

## Inspect the steps required for translations

Rather than actually converting `g` into other formats, let’s ask Metagraph how it will do the conversion. Each conversion requires a translator (written by plugin developers) to convert between the two formats. However, even if there isn’t a direct translator between two formats, Metagraph will find a path and take several translation steps as needed to perform the task.

The mechanism for viewing the plan is to invoke the translation from ``res.plan.translate`` rather than ``res.translate``. Other than the additional ``.plan``, the call signature is identical.

In this first example, there is a direct function which translates between `NetworkXGraphType` and `ScipyGraphType`.

In [ ]:
res.plan.translate(g, res.types.Graph.ScipyGraphType)

---
In this next example, there is no direct function which convert `NetworkXGraphType` into a `GrblasGraphType`. Instead, we have to first convert to `ScipyGraphType` and then to `GrblasGraphType` before finally arriving at our desired format.

While Metagraph will do the conversion automatically, understanding the steps involved helps users plan for expected computation time and memory usage. If needed, plugin developers can write a plugin to provide a direct translation path. 

In [ ]:
res.plan.translate(g, res.types.Graph.GrblasGraphType)

## Algorithm Example #1: Breadth First Search

Algorithms are described initially in an abstract definition. For bfs_iter, we take a `Graph` and return a `Vector` indicating the NodeIDs in the order visited.

After the abstract definition is written, multiple concrete implementations are written to operate on concrete types.

Let's look at the signature and specific implementations available for bfs_iter.

In [ ]:
res.algos.traversal.bfs_iter.signatures

We see that there are two implementations available, each with a different type of input graph.

---
Let's perform a breadth-first search with our different representations of `g`. We should get approximately the same answer no matter which implementation is chosen (same NodeIDs within each depth level of the traversal).

In [ ]:
cc = res.algos.traversal.bfs_iter(g, 0)
cc.value

In [ ]:
cc2 = res.algos.traversal.bfs_iter(g2, 0)
cc2.value

---
Similar to how we can view the plan for translations, we can view the plan for algorithms.

No translation is needed because we already have a concrete implementation which takes a `NetworkXGraph` as input.

In [ ]:
res.plan.algos.traversal.bfs_iter(g, 0)

---
In the next example, `g2` also satisfies a concrete implementation, so no input translation is required.

In [ ]:
res.plan.algos.traversal.bfs_iter(g2, 0)

## Algorithm Example #2: Pagerank

Let's look at the same pieces of information, but for pagerank. Pagerank takes a `Graph` and returns a `NodeMap` indicating the rank value of each node in the graph.

First, let's verify the signature and the implementations available.

We see that there are two implementations available, taking a `NetworkXGraph` or `GrblasGraph` as input.

In [ ]:
res.algos.centrality.pagerank.signatures

---
Let's look at the steps required in the plan if we start with a `ScipyGraph`. Then let's perform the computation.

We see that the `ScipyGraph` will need to be translated to a `GrblasGraph` in order to call the algorithm. **Metagraph will do this for us automatically.**

In [ ]:
res.plan.algos.centrality.pagerank(g2)

In [ ]:
pr = res.algos.centrality.pagerank(g2)
pr

The result is a `GrblasNodeMap` which can be inspected by looking at the underlying `.value`.

In [ ]:
pr.value

Let's translate it to a numpy array.

In [ ]:
pr_array = res.translate(pr, res.types.NodeMap.NumpyNodeMapType)
pr_array.value

Now let's verify that we get the same answer with the NetworkX implementation of Pagerank. 
We can ensure the NetworkX implementation is called by passing in a NetworkXGraph. Because no translations
are required, it will choose that implementation.

The result is a `PythonNodeMap`, which is simply a wrapper around a `dict`.

In [ ]:
pr2 = res.algos.centrality.pagerank(g)
pr2.value

Translate to a numpy array and verify the same results (within tolerance)

In [ ]:
pr2_array = res.translate(pr2, res.types.NodeMap.NumpyNodeMapType)
pr2_array.value

In [ ]:
abs(pr2_array.value - pr_array.value) < 1e-15